In [1]:
import cv2
import pickle
import os.path
import numpy as np
from tensorflow.keras import backend
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import np_utils
from resize_image import resize_image
from tensorflow.keras.models import Sequential


Using TensorFlow backend.


In [2]:
LETTER_IMAGES_FOLDER = "przetworzone_obrazy"
MODEL_FILENAME = "model_nn.hdf5"
MODEL_LABELS_FILENAME = "model_labels.dat"

IMAGE_SIZE_X = 20
IMAGE_SIZE_Y = 20

data = []
labels = []

In [3]:

for image_file in paths.list_images(LETTER_IMAGES_FOLDER):
  
    # Odczyt pliku za pomoca opencv 
    image = cv2.imread(image_file)
    # Sprawdzenie czy udalo sie zaladowac plik
    try:
        if image.size == 0 :
            continue
    except:
        print("Not loaded")
        continue
    # Przetworzenie obrazu z RBG do odcieni szarosci
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    

    # Skalowanie zdjecia do wielkosci 20x20
    image = resize_image(image, IMAGE_SIZE_X, IMAGE_SIZE_Y)

    # Przetworzenie zdjecia tak aby ostatecznie znajdowalo sie w 3 wymiarach (20,20,1)
    image = np.expand_dims(image, axis=2)

    # Pobranie nazwy pliku ze sciezki 
    label = image_file.split(os.path.sep)[-2]

    # Dodanie obrazu do listy 
    data.append(image)

    # Dodanie etykiety do listy
    labels.append(label)
    

In [4]:
# Normalizacja pikseli
data = np.array(data, dtype="float") / 255.0

# Przekstałcenie etykiet do tablicy z numpy
labels = np.array(labels)

# Podział danych na uczace i treningowe
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.33, random_state=42)
    


In [5]:
# Przeksztalcenie etykiet do listy i zapis modelu etykiet do pliku
lb = LabelBinarizer().fit(Y_train)
Y_train = lb.transform(Y_train)
Y_test = lb.transform(Y_test)

with open(MODEL_LABELS_FILENAME, "wb") as f:
    pickle.dump(lb, f)

# Tworzenie modelu sieci neuronowej
model = Sequential()
model.add(Conv2D(20, (5, 5), padding="same", input_shape=(IMAGE_SIZE_X, IMAGE_SIZE_Y, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(500, activation="relu"))
model.add(Dense(32, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Trenowanie seci
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=5, verbose=1)

Train on 18784 samples, validate on 9252 samples
Epoch 1/5
18784/18784 [==============================] - 9s 475us/sample - loss: 0.1974 - accuracy: 0.9565 - val_loss: 0.0915 - val_accuracy: 0.9749
Epoch 2/5
18784/18784 [==============================] - 11s 561us/sample - loss: 0.0132 - accuracy: 0.9980 - val_loss: 0.0192 - val_accuracy: 0.9974
Epoch 3/5
18784/18784 [==============================] - 8s 412us/sample - loss: 0.0064 - accuracy: 0.9985 - val_loss: 0.0124 - val_accuracy: 0.9986
Epoch 4/5
18784/18784 [==============================] - 8s 405us/sample - loss: 0.0037 - accuracy: 0.9995 - val_loss: 0.0120 - val_accuracy: 0.9990
Epoch 5/5
18784/18784 [==============================] - 8s 406us/sample - loss: 0.0030 - accuracy: 0.9996 - val_loss: 0.0131 - val_accuracy: 0.9988


In [6]:
# Zapis modelu sieci do pliku
model.save(MODEL_FILENAME)